GOALS:
<li>Implementation of dynamic programming algorithms such as policy iteration and value iteration.</li>
<li>Applying dynamic programming algorithms to the Tic Tac Toe game to find the optimal strategie. </li>
<li>Comparing results. </li>

### <font color='red'> 1. Defining the environment,states, actions and rewards </font>

In order to implement a Tic Tac Toe game where a player can play against the computer, we will consider the problem as an MDP. We will define its principal components such as the environment, states, actions, and rewards and then apply dynamic programming algorithms.

##### <font color='green'> 1. States </font>

We define a state in our MDP model as a board configuration of the game. Our goal is to let the agent behave optimally while playing with 'O' or 'X'. We choose to train our agent for 'O', and then we can deal with 'X' case by exchanging roles. <br>
We represent a state as a 9-character string (a string with a length of 9) containing 'X','O' and ' ' for empty cases.
We put all the states in an array, which contains all possible states that an agent can take an action in and some terminal actions, conditional on the agent playing with 'O'.

  First, we define a function that takes a state and decides whether this state is valid or not.

In [1]:
def is_valid_board(board):
    # Check if the board has exactly 9 characters
    if len(board) != 9:
        return False

    # Count the number of Xs and Os
    count_x = board.count('X')
    count_o = board.count('O')

    # Check if the difference in counts of X and O is at most 1
    if abs(count_x - count_o) > 1:
        return False

    # Check if the number of O is than X
    if count_o > count_x :
        return False

    # Check for invalid characters
    valid_characters = set(['X', 'O', ' '])
    if any(char not in valid_characters for char in board):
        return False

    # Check if the game is already won by one of the players
    if is_winner(board, 'X') and is_winner(board, 'O'):
        return False

    return True

def is_winner(board, player):
    # Check rows, columns, and diagonals for a win
    win_conditions = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Rows
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columns
        [0, 4, 8], [2, 4, 6]              # Diagonals
    ]

    for condition in win_conditions:
        if all(board[i] == player for i in condition):
            return True

    return False

Next, we create a function that can recursively generate all possible 9-character strings ($3^9$ states) containing 'X', 'O', and ' ', while preserving only valid states (boards) using the "is_valid_board" function.

In [2]:
# Generate all valid board configurations
valid_states = set()
empty_board = '         '  # 9 empty spaces

def generate_states(board, index=0):
    if index == 9:
        if is_valid_board(board):
            valid_states.add(board)
        return

    # Recursively try all possible moves for X and O
    for char in 'XO ':
        new_board = board[:index] + char + board[index + 1:]
        generate_states(new_board, index + 1)

generate_states(empty_board)

# Convert the set of valid states into a list
valid_states_list = list(valid_states)

print("Number of valid states is : ",len(valid_states_list))

Number of valid states is :  5890


These 5890 states involve all possible states that a player (supposed to play with 'O') can encounter in a real game, including some terminal states (end game scenarios) such as a winning case or a draw when the numbers of 'O' and 'X' are equal on the board. These are useful states that will be used later for evaluating the rest of the non-terminal states.

#### <font color='green'> 2. Actions </font>

An action is represented as an integer comprising a value between 0 and 8. This integer is the index of an empty space in the 9-characters string (on the board).

#### <font color='green'> 3. Environment </font>

We define our game environment with a class called TicTacToe. This class has two attributes and some useful functions to ensure efficient gameplay while maintaining the rules and logic of the Tic Tac Toe game.

In [3]:
class TicTacToe:
    def __init__(self, initial_board=None):
        if initial_board is None:
            self.board = [' ' for _ in range(9)]  # Initialize an empty 3x3 grid
        else:
            # Verify that the provided initial_board is a valid 9-character string
            if len(initial_board) == 9 and all(char in ('X', 'O', ' ') for char in initial_board):
                self.board = list(initial_board)
            else:
                raise ValueError("Invalid initial board configuration")

        self.current_player = 'X'  # Start with player 'X'
    def define_current_player(self,player) :
        self.current_player=player

    def get_board_config(self):
        return "".join(self.board)

    def set_board_from_config(self,config):
        for i in range(9):
            self.board[i]=config[i]
        return True

    def display_board(self):
        for i in range(3):
            for j in range(3):
                print(self.board[i * 3 + j], end="")

                if j < 2:
                    print(" | ", end="")

            print()  # Move to the next row
            if i < 2:
                print("---------")  # Display horizontal dividers between rows

    def get_all_possible_next_states(self,player):
        empty_cells = [i for i, char in enumerate(self.board) if char == ' ']
        next_states = []
        for cell in empty_cells:
            next_state = self.board.copy()
            next_state[cell] = player
            next_states.append("".join(next_state))
        return next_states

    def get_possible_moves(self):
        empty_cells=[]
        game,winner=self.is_game_over()
        if game!=True :
            for i in range(9):
                if self.board[i]==' ' :
                    empty_cells.append(i)
            return empty_cells
        return None
    def make_move(self, position):
        if self.board[position] == ' ':
            self.board[position] = self.current_player
            # Toggle the current player for the next turn
            self.current_player = 'X' if self.current_player == 'O' else 'O'
            return True
        else:
            print("Invalid move. Try again. ")
            return False

    def is_game_over(self):
        # Check for win conditions
        # Check rows, columns, and diagonals for three in a row
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columns
            [0, 4, 8], [2, 4, 6]  # Diagonals
        ]

        for condition in win_conditions:
            a, b, c = condition
            if (
                self.board[a] == self.board[b] == self.board[c] != ' '
            ):
                return True, self.board[a]

        # Check for a draw (no empty spaces left)
        if ' ' not in self.board:
            return True, None  # Game is a draw

        return False, None  # Game is not over yet

#### <font color='green'> 4. Rewards </font>

For the rewards we define a function that takes a state and return its reward as follow :

In [4]:
def Reward(board_config) :
    TicTacToeEnv=TicTacToe(board_config)
    game,winner=TicTacToeEnv.is_game_over()
    if winner=='O' :
        return 500
    elif winner=='X' :
        return -3500
    else :
        return 0

### <font color='red'> 2. Policy iteration implementation  </font>

Our goal in the Policy Iteration algorithm is to find an optimal policy that maps each state to the optimal actions using the calculation of the value function. <br><br>
Initially, an arbitrary policy is initialized, and the value function of all states is set to 0.



In [5]:
# Initialize the value function for each state to 0
value_function = {state: 0 for state in valid_states_list}

policy={} # Initialize an empty dictionary for the policy

for state in valid_states_list:
    legal_actions = [i for i, char in enumerate(state) if char == ' ']
    if is_winner(state,'X') or is_winner(state,'O') :
        policy[state]=None  # If the current state is a winning state, set the policy to None (No action to be taken)
    else :
        legal_actions = [i for i, char in enumerate(state) if char == ' ']
        if legal_actions:
            policy[state] = legal_actions[0]
        else :                      # If there are no legal actions available, set the policy to None (draw state)
            policy[state] = None

To implement the policy iteration algorithm, we first need to implement two functions:

<li>Policy evaluation: This function evaluates the current policy given as input and returns its value function.</li>
<li>Policy improvement: This function takes a policy and its value function as input and returns a better policy than the one given.</li>

In [6]:
def policy_evaluation(policy, value_function):
    tic_tac_toe_env = TicTacToe()
    gamma = 0.8
    delta = 1e-8  # A small threshold for convergence
    num_iteration=0
    while True:
        delta = 0
        for board_config in value_function:
            old_value = value_function[board_config]
            tic_tac_toe_env.set_board_from_config(board_config)

            action = policy[board_config]

            if action is not None:  # Only update non-terminal states with legal actions
                tic_tac_toe_env.define_current_player('O')
                tic_tac_toe_env.make_move(action)
                board_config_after_move=tic_tac_toe_env.get_board_config()
                successor_states = tic_tac_toe_env.get_all_possible_next_states("X")

                # Initialize the new value for this state
                new_value = 0

                # Check if the current state is a terminal state (win or draw)
                if is_winner(board_config_after_move, 'X') or is_winner(board_config_after_move, 'O') or ' ' not in board_config_after_move:
                    # Set the value to the reward of the terminal state
                    new_value = Reward(board_config_after_move)
                else:
                    # Iterate over all possible successor states
                    num_successor_states = len(successor_states)

                    #we suppose that the envirement responds uniformly random,
                    #so for each next state s' : p(s'|s,a)=1/(Number of possible moves by the opponent.)
                    for successor_config in successor_states:
                        new_value += (1.0 / num_successor_states) * (Reward(successor_config) + gamma * value_function[successor_config])

                value_function[board_config] = new_value
            delta = max(delta, abs(old_value - value_function[board_config]))
        num_iteration+=1
        if delta < 1e-8:
            return num_iteration

In [7]:
def policy_improvement(policy, value_function):
    tic_tac_toe_env=TicTacToe()
    gamma=0.8
    policy_stable = True
    for board_config in policy:  # Iterate over all possible board configurations
        old_action = policy[board_config]
        if old_action is not None:  # Skip Terminal state
            best_action = None
            best_value = -float('inf')
            tic_tac_toe_env.set_board_from_config(board_config)
            for action in tic_tac_toe_env.get_possible_moves():
                # Make a move and calculate the expected value
                tic_tac_toe_env.define_current_player('O')
                tic_tac_toe_env.make_move(action)
                board_config_after_move=tic_tac_toe_env.get_board_config()
                successor_states = tic_tac_toe_env.get_all_possible_next_states("X")
                 # Initialize the new value for this state
                expected_value = 0
                # Check if the current state is a terminal state (win or draw)

                if is_winner(board_config_after_move, 'X') or is_winner(board_config_after_move, 'O') or ' ' not in board_config_after_move:
                    # Set the value to the reward of the terminal state
                    expected_value = Reward(board_config_after_move)
                else:
                    # Iterate over all possible successor states
                    num_successor_states = len(successor_states)
                    for successor_config in successor_states:
                        expected_value += (1.0 / num_successor_states) * (Reward(successor_config) + gamma * value_function[successor_config])


                if expected_value > best_value:
                    best_action = action
                    best_value = expected_value
                tic_tac_toe_env=TicTacToe(board_config)

            policy[board_config] = best_action

        if old_action != policy[board_config]:
            policy_stable = False

    return policy_stable

Now we define the policy iteration function that takes an arbitray policy and return the optimal policy

In [8]:
def policy_iteration(policy, value_function):
    num_iter=0
    policy_stable = False
    while not policy_stable:

        # Policy evaluation
        policy_evaluation(policy,value_function)

        # Policy improvement
        policy_stable = policy_improvement(policy,value_function)
        value_function = {state: 0 for state in valid_states_list}
        # Update the value function
        num_iter+=1
    print("The number of iterations needed to converge is : ",num_iter)
    return policy

We execute policy_iteration to get the optimal policy

In [9]:
num_iter=policy_iteration(policy,value_function)

The number of iterations needed to converge is :  4


Results  :

To test the performance of the policy we got, we create a function named game_tester. This function takes a policy and the number of games as arguments, then simulates the desired number of games and returns the win rate, loss rate, and draw rate

In [10]:
import random

def game_tester(policy,turn, number_of_games=1000):
    win_count = 0
    loss_count = 0
    draw_count = 0
    player_turn=turn
    for _ in range(number_of_games):
        tic_tac_toe_env = TicTacToe()
        game_over = False

        while not game_over:
            current_player = 'X' if player_turn % 2 == 0 else 'O'

            if current_player == 'X':
                possible_moves = tic_tac_toe_env.get_possible_moves()
                move = random.choice(possible_moves)
            else:
                move = policy[tic_tac_toe_env.get_board_config()]

            tic_tac_toe_env.define_current_player(current_player)
            tic_tac_toe_env.make_move(move)
            player_turn += 1

            game_over, winner = tic_tac_toe_env.is_game_over()
        player_turn=turn
        if winner == 'X':
            loss_count += 1
        elif winner == 'O':
            win_count += 1
        else:
            draw_count += 1

    rate_win = win_count / number_of_games
    rate_loss = loss_count / number_of_games
    rate_draw = draw_count / number_of_games

    print("Win Rate: {:.2%}".format(rate_win))
    print("Loss Rate: {:.2%}".format(rate_loss))
    print("Draw Rate: {:.2%}".format(rate_draw))

# Call the game_tester function
print("if the oppenent starts first : ")
game_tester(policy,0,number_of_games=100000)
print("if the agent starts first : ")
game_tester(policy,1,number_of_games=100000)

if the oppenent starts first : 
Win Rate: 91.62%
Loss Rate: 0.00%
Draw Rate: 8.38%
if the agent starts first : 
Win Rate: 99.47%
Loss Rate: 0.00%
Draw Rate: 0.53%


To simulate a game between a human and an agent, we create a 'launch_game' function that takes a policy defining how the agent will behave as an argument and launches a game.

In [11]:
def launch_game(policy,turn_count=0):
    tic_tac_toe_env = TicTacToe()
    game_over = False

    while not game_over:
        current_player = 'X' if turn_count % 2 == 0 else 'O'
        if current_player=='X' :
            print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
            print("Your turn : \n")
            tic_tac_toe_env.display_board()

        if current_player == 'X':
            move = int(input("Enter a position (between 0 and 8) : "))
        else:
            move = policy[tic_tac_toe_env.get_board_config()]

        tic_tac_toe_env.define_current_player(current_player)
        if tic_tac_toe_env.make_move(move):
            turn_count += 1

        game_over, winner = tic_tac_toe_env.is_game_over()
    if winner == 'X':
        print("You won!\n\n")
    elif winner == 'O':
        print("You lost!\n\n")
    else:
        print("It's a draw!\n\n")
    tic_tac_toe_env.display_board()


In [12]:
#lanch_game(policy)

### <font color='red'> 3. Value iteration implementation  </font>

In [13]:
def value_iteration(policy,value_function):
    gamma = 0.9  # Discount factor
    delta_threshold = 1e-8  # A small threshold for convergence
    num_iter=0
    while True:
        delta = 0
        for board_config in value_function:
            old_value = value_function[board_config]

            best_value = -float('inf')
            tic_tac_toe_env = TicTacToe()
            tic_tac_toe_env.set_board_from_config(board_config)

            if policy[board_config]!=None :
                for action in tic_tac_toe_env.get_possible_moves():
                    # Make a move and calculate the expected value
                    tic_tac_toe_env.define_current_player('O')
                    tic_tac_toe_env.make_move(action)
                    board_config_after_move = tic_tac_toe_env.get_board_config()

                    successor_states = tic_tac_toe_env.get_all_possible_next_states("X")

                    # Initialize the new value for this state
                    expected_value = 0

                    # Check if the current state is a terminal state (win or draw)
                    if is_winner(board_config_after_move, 'X') or is_winner(board_config_after_move, 'O') or ' ' not in board_config_after_move:
                        # Set the value to the reward of the terminal state
                        expected_value = Reward(board_config_after_move)
                    else:
                        # Iterate over all possible successor states
                        num_successor_states = len(successor_states)
                        for successor_config in successor_states:
                            expected_value += (1.0 / num_successor_states) * (
                                    Reward(successor_config) + gamma * value_function[successor_config])

                    if expected_value > best_value:
                        best_value = expected_value
                    tic_tac_toe_env=TicTacToe(board_config)
                value_function[board_config] = best_value
                delta = max(delta, abs(old_value - value_function[board_config]))
        num_iter+=1
        if delta < delta_threshold:
            break

    # Policy improvement after value iteration
    policy_improvement(policy,value_function) #after calculating vpi* we execute policy improvement for choosing the greedy policy
    print("The number of iterations needed to converge is : ",num_iter)
    return policy


Initialization

In [14]:
# Initialize the value function for each state to 0
value_function = {state: 0 for state in valid_states_list}

policy_2={} # Initialize an empty dictionary for the

for state in valid_states_list:
    legal_actions = [i for i, char in enumerate(state) if char == ' ']
    if is_winner(state,'X') or is_winner(state,'O') :
        policy_2[state]=None  # If the current state is a winning state, set the policy to None (No action to be taken)
    else :
        legal_actions = [i for i, char in enumerate(state) if char == ' ']
        if legal_actions:
            policy_2[state] = legal_actions[0]
        else :                      # If there are no legal actions available, set the policy to None (draw state no ation to be taken)
            policy_2[state] = None

Train policy_2 using the value iteration function

In [15]:
n=value_iteration(policy_2,value_function)

The number of iterations needed to converge is :  5


In [16]:
# Call the game_tester function
print("if the oppenent starts first : ")
game_tester(policy_2,0,number_of_games=100000)
print("if the agent starts first : ")
game_tester(policy_2,1,number_of_games=100000)

if the oppenent starts first : 
Win Rate: 91.65%
Loss Rate: 0.00%
Draw Rate: 8.35%
if the agent starts first : 
Win Rate: 99.49%
Loss Rate: 0.00%
Draw Rate: 0.51%


In the end, we will save our agent's policy in a JSON file for future use. The agent is currently trained to use 'O' optimally, but we also want to save an agent that can use 'X' optimally. To achieve this, we will exchange roles between 'X' and 'O' in each state while preserving the same actions.

In [17]:
import json

def convert_state(state):
    s=list(state)
    for i in range(len(state)):
        if state[i]=='X' :
            s[i]='O'
        elif state[i]=='O' :
            s[i]='X'
        else :
            s[i]=state[i]
    return "".join(s)

policy_3={convert_state(state) : policy_2[state] for state in policy_2}

# Saving the dictionary to a file
with open('Optimal_strategie_DP.json', 'w') as f:
    json.dump(policy_3, f)

### <font color='red'> 4. Conclusion:  </font>

Value iteration and policy iteration are both efficient algorithms that quickly converge, making them valuable for reinforcement learning, including in complex games like Tic-Tac-Toe.